# **Introdução: Análise de Atrasos em Voos nos EUA**

Neste projeto, vou analisar os **atrasos de voos** nos Estados Unidos, utilizando dados provenientes do **Kaggle**. O conjunto de dados original contém informações sobre os voos, incluindo dados sobre **atrasos de chegada** e **decolagem**, **companhias aéreas**, **aeroportos de origem e destino**, e **informações sobre o voo**.

A proposta é construir um pipeline de dados utilizando tecnologias na nuvem. O pipeline irá envolver a busca, coleta, modelagem, carga e análise dos dados.

### **Fontes dos Dados**:
Os dados utilizados neste projeto foram extraídos de Kaggle e incluem os seguintes arquivos:
- **`flights.csv`**: Contém informações sobre os voos, incluindo os atrasos.
- **`airlines.csv`**: Contém informações sobre as companhias aéreas.
- **`airports.csv`**: Contém informações sobre os aeroportos de origem e destino.

Esses dados foram carregados no **Databricks File System (DBFS)** e foram processados utilizando a plataforma Databricks para realizar **transformações e análises**.

### **Objetivo**:
O objetivo deste projeto é realizar uma análise exploratória sobre os **atrasos de voos**, utilizando os dados das três tabelas mencionadas.

## **Perguntas de Negócio**
- Quais companhias aéreas apresentam maior proporção de voos com atraso?
- Qual é o percentual de voos cancelados por companhia aérea?
- A distância ou o tempo de voo influenciam o atraso na chegada?
- Quais aeroportos concentram mais atrasos na decolagem e/ou chegada?

Essas perguntas guiarão tanto a análise exploratória quanto a modelagem preditiva, contribuindo para a identificação de padrões relevantes que expliquem os atrasos nos voos.

# **Execução em Nuvem**

Nessa seção documentarei o passo a passo executado em nuvem no Databricks para fazer a análise.

## **Coleta de Dados**

Para este projeto, o dataset **2015 Flight Delays and Cancellations** foi carregado diretamente para o **Databricks File System (DBFS)**. O arquivo foi baixado do Kaggle e armazenado no DBFS para ser utilizado nas etapas de processamento e análise.

### **Tabela Fato**

O código abaixo mostra como a **Tabela Fato**, o arquivo `flights-1.csv` foi carregado para ser utilizado no projeto.

![Dados Carregados](Evidence/2_dataset_loaded.png)


### **Tabelas Dimensão**

O código abaixo mostra como as **Tabelas Dimensão**, os arquivos`airports-1.csv` e `airlines-1.csv` foram carregadas para serem utilizadas no projeto.

![Data Loaded](Evidence/2_dimension_tables_loaded.png)

## **ETL (Extração, Transformação e Carga de Dados)**

Na seção de ETL, realizei as transformações dos dados para prepará-los para a análise. Comecei com a limpeza e a junção das tabelas de aeroportos e companhias aéreas com a Tabela Fato. A seguir, apliquei as transformações necessárias para responder às perguntas de negócio, como calcular a proporção de voos com atraso por companhia aérea, os atrasos por aeroporto, e a influência da distância e do tempo de voo nos atrasos.



### **Limpeza dos dados e junção das tabelas**

Iniciei com a limpeza dos dados, removendo espaços em branco das colunas de junção, e então preparei as tabelas dimensionais. Realizei a união entre as tabelas de aeroportos e companhias aéreas, e a Tabela Fato, conforme os critérios definidos.

O código a seguir faz a limpeza e a junção das tabelas de aeroportos e companhias aéreas com a Tabela Fato.

![limpeza](Evidence/limpeza.png)

### **Calculando Atrasos por companhia aérea**

O código executado para calcular a proporção de atrasos por companhia aérea pode ser visto no print abaixo, onde a **proporção de voos com atraso** por **companhia aérea** foi calculada.

![atraso_companhia](Evidence/atraso_companhia.png)

### **Atrasos por Aeroporto**

Nesta etapa, analisei os **atrasos por aeroporto**, tanto no **aeroporto de origem** quanto no **aeroporto de destino**. Para isso, filtrei os voos com atraso tanto na **decolagem** quanto na **chegada**, e realizei uma contagem dos voos atrasados por cada aeroporto.

O código executado para calcular os **atrasos por aeroporto de origem** e **atrasos por aeroporto de destino** pode ser visto abaixo, onde os voos com atraso na **decolagem** são contados para o **aeroporto de origem** e os voos com atraso na **chegada** são contados para o **aeroporto de destino**.


![atraso_aeroporto](Evidence/atraso_aeroporto.png)

### **Influência da Distância e Tempo de Voo no Atraso**

Nesta etapa, buscamos entender como a **distância** e o **tempo de voo** afetam os **atrasos na chegada** dos voos. Para isso, realizei o cálculo da **correlação** entre **`DISTANCE`** (distância do voo), **`AIR_TIME`** (tempo de voo) e **`ARRIVAL_DELAY`** (atraso na chegada).

Primeiro, garanti que as colunas envolvidas na correlação fossem **numéricas**. Para isso, utilizei a função **`cast("float")`** para converter as colunas **`DISTANCE`**, **`AIR_TIME`** e **`ARRIVAL_DELAY`** para **tipos numéricos**. Depois, calculamos a **correlação** entre essas variáveis.

O código executado para calcular as correlações pode ser visto abaixo.

![influencia](Evidence/influencia.png)

### **Percentual de Voos Cancelados por Companhia Aérea**

Nesta etapa, calculei o **percentual de voos cancelados** por **companhia aérea**. Para isso, primeiro filtrei os voos **cancelados** e contei o número de voos cancelados por **companhia aérea**. Em seguida, calculei o **percentual de cancelamento** dividindo o número de voos cancelados pelo número total de voos de cada **companhia aérea**.

O código executado para calcular o percentual de voos cancelados por companhia aérea pode ser visto abaixo.

![perentual_companhia](Evidence/percentual_companhia.png)

## **Catálogo de Dados**

O catálogo de dados descreve as colunas, os tipos de dados e os valores possíveis nas tabelas que compõem o Data Lake do projeto. Abaixo estão os detalhes das tabelas e colunas com dados reais extraídos e transformados.



### **Colunas Presentes**


![catalogo1](Evidence/catalogo_1.png)

### **Capturando valores máximos e mínimos**

![catalogo2](Evidence/catalogo_2.png)

### **Exibindo colunas categoricas**

![catalogo3](Evidence/catalogo_3.png)

### **Catálogo de Dados Completo**


| Coluna                      | Tipo de Dados | Descrição                                                                                        | Exemplos                          | Valores Mínimos e Máximos               | Categorias Possíveis                   |
|-----------------------------|---------------|--------------------------------------------------------------------------------------------------|-----------------------------------|------------------------------------------|-----------------------------------------|
| `YEAR`                       | String        | Ano do voo.                                                                                        | 2015                              | Mínimo: 2015, Máximo: 2021               | Não se aplica                           |
| `MONTH`                      | String        | Mês do voo.                                                                                       | 1, 2, 3                           | Mínimo: 1, Máximo: 12                    | Não se aplica                           |
| `DAY`                        | String        | Dia do mês.                                                                                        | 1, 2, 3                           | Mínimo: 1, Máximo: 31                    | Não se aplica                           |
| `DAY_OF_WEEK`                | String        | Dia da semana em formato numérico (1=Domingo, 7=Sábado).                                           | 4                                 | 1 - 7                                    | Não se aplica                           |
| `AIRLINE`                    | String        | Código da companhia aérea.                                                                         | UA, NK, AA, etc.                  | Não aplicável                           | Códigos das companhias aéreas           |
| `FLIGHT_NUMBER`              | String        | Número do voo.                                                                                   | 98, 2336, 840                     | Não aplicável                           | Não se aplica                           |
| `TAIL_NUMBER`                | String        | Número da aeronave.                                                                                | N407AS, N3KUAA                    | Não aplicável                           | Não se aplica                           |
| `ORIGIN_AIRPORT`             | String        | Código do aeroporto de origem.                                                                    | ANC, LAX, SFO                     | Não aplicável                           | Códigos de aeroporto (IATA)            |
| `DESTINATION_AIRPORT`        | String        | Código do aeroporto de destino.                                                                   | SEA, PBI, CLT                     | Não aplicável                           | Códigos de aeroporto (IATA)            |
| `SCHEDULED_DEPARTURE`        | String        | Hora programada de partida.                                                                       | 0005, 0010, 0020                  | Não aplicável                           | Não se aplica                           |
| `DEPARTURE_TIME`             | String        | Hora real de partida.                                                                             | 2354, 0002, 0018                  | Não aplicável                           | Não se aplica                           |
| `DEPARTURE_DELAY`            | String        | Atraso na partida (em minutos).                                                                   | -11, -8, -2                       | Mínimo: -60, Máximo: 1000               | Não se aplica                           |
| `TAXI_OUT`                   | String        | Tempo de táxi até a decolagem (em minutos).                                                       | 1, 99                             | Mínimo: 0, Máximo: 180                  | Não se aplica                           |
| `WHEELS_OFF`                 | String        | Hora em que a aeronave decola.                                                                    | 0015, 0014, 0034                  | Não aplicável                           | Não se aplica                           |
| `SCHEDULED_TIME`             | String        | Tempo programado de voo (em minutos).                                                             | 100, 99, 120                      | Mínimo: 30, Máximo: 1500                | Não se aplica                           |
| `ELAPSED_TIME`               | String        | Tempo total de voo (em minutos).                                                                  | 194, 279, 293                     | Mínimo: 30, Máximo: 1500                | Não se aplica                           |
| `AIR_TIME`                   | Float         | Tempo real de voo (em minutos).                                                                   | 169.0                             | Mínimo: 10, Máximo: 1500                | Não se aplica                           |
| `DISTANCE`                   | Float         | Distância percorrida (em milhas).                                                                  | 1448.0, 2330.0, 2296.0            | Mínimo: 100, Máximo: 3000               | Não se aplica                           |
| `WHEELS_ON`                  | String        | Hora em que a aeronave aterrissa.                                                                 | 0404, 0737, 0800                  | Não aplicável                           | Não se aplica                           |
| `TAXI_IN`                    | String        | Tempo de táxi após a aterrissagem (em minutos).                                                   | 4, 8, 11                          | Mínimo: 0, Máximo: 60                   | Não se aplica                           |
| `SCHEDULED_ARRIVAL`          | String        | Hora programada de chegada.                                                                       | 0430, 0750, 0806                  | Não aplicável                           | Não se aplica                           |
| `ARRIVAL_TIME`               | String        | Hora real de chegada.                                                                             | 0408, 0741, 0811                  | Não aplicável                           | Não se aplica                           |
| `ARRIVAL_DELAY`              | Float         | Atraso na chegada (em minutos).                                                                   | -22.0, -9.0, 5.0                  | Mínimo: -60, Máximo: 1000               | Não se aplica                           |
| `DIVERTED`                   | String        | Indica se o voo foi desviado (0=Não, 1=Sim).                                                      | 0, 1                              | Não aplicável                           | 0, 1                                     |
| `CANCELLED`                  | String        | Indica se o voo foi cancelado (0=Não, 1=Sim).                                                     | 0, 1                              | Não aplicável                           | 0, 1                                     |
| `CANCELLATION_REASON`        | String        | Razão do cancelamento (se aplicável).                                                             | null                              | Não aplicável                           | Não se aplica                           |
| `AIR_SYSTEM_DELAY`           | String        | Atraso devido ao sistema aéreo (em minutos).                                                      | null                              | Mínimo: 0, Máximo: 120                  | Não se aplica                           |
| `SECURITY_DELAY`             | String        | Atraso devido a questões de segurança (em minutos).                                               | null                              | Mínimo: 0, Máximo: 120                  | Não se aplica                           |
| `AIRLINE_DELAY`              | String        | Atraso devido à companhia aérea (em minutos).                                                     | null                              | Mínimo: 0, Máximo: 120                  | Não se aplica                           |
| `LATE_AIRCRAFT_DELAY`        | String        | Atraso devido a aeronave atrasada (em minutos).                                                   | null                              | Mínimo: 0, Máximo: 120                  | Não se aplica                           |
| `WEATHER_DELAY`              | String        | Atraso devido ao clima (em minutos).                                                             | null                              | Mínimo: 0, Máximo: 120                  | Não se aplica                           |
| `ORIGIN_AIRPORT_NAME`        | String        | Nome do aeroporto de origem.                                                                      | Ted Stevens Anchorage, LAX         | Não aplicável                           | Nomes de aeroportos                     |
| `DESTINATION_AIRPORT_NAME`   | String        | Nome do aeroporto de destino.                                                                     | Seattle-Tacoma, Palm Beach Intl    | Não aplicável                           | Nomes de aeroportos                     |
| `AIRLINE_NAME`               | String        | Nome da companhia aérea.                                                                          | Alaska Airlines Inc., American Airlines Inc. | Não aplicável                   | Nomes das companhias aéreas             |

## **Análise Exploratória**

Nesta etapa, realizamos uma análise exploratória simples dos dados para responder as perguntas de negócio:

- Quais companhias aéreas apresentam maior proporção de voos com atraso?
- Qual é o percentual de voos cancelados por companhia aérea?
- A distância ou o tempo de voo influenciam o atraso na chegada?
- Quais aeroportos concentram mais atrasos na decolagem e/ou chegada?


### **Quais companhias aéreas apresentam maior proporção de voos com atraso?**

Para responder a essa pergunta, calculei a **proporção de voos com atraso** por companhia aérea. A análise foi realizada comparando a quantidade de voos atrasados com o total de voos realizados por cada companhia aérea.

![proporcao_atraso](Evidence/proporcao_atraso.png)

**Análise dos Resultados:**

- **Companhias com maior proporção de atrasos:**
   - **NK** (Norwegian Air) apresenta a maior proporção de voos com atraso, com cerca de **48.46%** dos seus voos atrasados.
   - **F9** (Spirit Airlines) também mostra uma alta proporção, com **45.39%** de seus voos com atraso.

- **Companhias com menor proporção de atrasos:**
   - **DL** (Delta Air Lines) tem a menor proporção de atrasos, com **28.64%** de seus voos atrasados.
   - **AS** (Alaska Airlines) também apresenta uma proporção relativamente baixa, com **33.01%** de voos atrasados.

### **Qual é o percentual de voos cancelados por companhia aérea?**

Agora, vou calcular o **percentual de voos cancelados por companhia aérea**. Para isso, vou comparar o número de voos cancelados com o número total de voos realizados por cada companhia aérea. O objetivo é entender qual a proporção de voos cancelados por companhia aérea.

![voo_cancelado](Evidence/voo_cancelado.png)

**Análise dos Resultados**

- **Companhias com maior percentual de voos cancelados:**
   - **MQ** (Envoy Air) apresenta o maior percentual de voos cancelados, com cerca de **5.10%** dos seus voos cancelados. Isso pode indicar problemas operacionais ou baixa confiabilidade nas operações dessa companhia.
   - **US** (US Airways) também apresenta um percentual elevado, com **2.05%** de voos cancelados.

- **Companhias com menor percentual de voos cancelados:**
   - **HA** (Hawaiian Airlines) tem o menor percentual de voos cancelados, com apenas **0.22%** de seus voos cancelados, indicando uma operação bastante estável.
   - **AS** (Alaska Airlines) também tem um percentual baixo de cancelamentos, com **0.39%** de seus voos cancelados.

**Conclusão:**
A análise do percentual de voos cancelados por companhia aérea revela que **MQ** e **US** têm os maiores percentuais de voos cancelados, o que pode indicar questões específicas dessas companhias em relação à confiabilidade operacional. Por outro lado, **HA** e **AS** demonstram um bom desempenho, com cancelamentos mínimos.

### **A distância ou o tempo de voo influenciam o atraso na chegada?**

Agora, vou verificar se há uma correlação entre **distância dos voos** e o **atraso na chegada** (ARRIVAL_DELAY). Além disso, também vou investigar se o **tempo de voo** (AIR_TIME) tem influência sobre o atraso na chegada. Para isso, calculei a **correlação entre as variáveis**.

![correlacao](Evidence/correlacao.png)

**Análise dos Resultados**

- **Correlação entre distância e atraso na chegada:**
   - A correlação entre **distância** e **atraso na chegada** é de **-0.02**, o que indica uma **correlação muito fraca e negativa**. Isso sugere que não há uma relação significativa entre a distância percorrida pelos voos e o atraso na chegada. Ou seja, voos mais longos ou mais curtos não têm uma influência clara no atraso.

- **Correlação entre tempo de voo e atraso na chegada:**
   - A correlação entre **tempo de voo** e **atraso na chegada** é de **-0.0038**, o que também indica uma **correlação extremamente fraca e negativa**. Isso sugere que o tempo de voo não tem uma relação substancial com os atrasos na chegada, sendo também uma variável com pouco impacto no atraso.

**Conclusão:**
Ambas as análises de correlação indicam que **nem a distância nem o tempo de voo** têm uma influência significativa sobre o **atraso na chegada**. Esses resultados sugerem que outros fatores podem ser mais determinantes para os atrasos, como condições climáticas, problemas operacionais ou outros fatores logísticos que não estão representados por essas variáveis.

### **Quais aeroportos concentram mais atrasos na decolagem e/ou chegada?**

Nesta análise, investiguei os **atrasos nos aeroportos de origem** e **nos aeroportos de destino**. O objetivo é identificar quais aeroportos têm o maior número de voos com atraso, tanto na decolagem quanto na chegada. Para isso, contei o número de voos com atraso para cada aeroporto e apresento os resultados.


![top5](Evidence/aeroportos_ruims.png)

**Análise dos Resultados**

- **Top 5 aeroportos com mais atrasos na decolagem (origem):**
1. **ATL (Hartsfield-Jackson Atlanta International)**: 129,484 atrasos
2. **ORD (Chicago O'Hare International)**: 121,706 atrasos
3. **DFW (Dallas/Fort Worth International)**: 96,475 atrasos
4. **DEN (Denver International)**: 89,209 atrasos
5. **LAX (Los Angeles International)**: 81,954 atrasos

- **Top 5 aeroportos com mais atrasos na chegada (destino):**
1. **ATL (Hartsfield-Jackson Atlanta International)**: 105,683 atrasos
2. **ORD (Chicago O'Hare International)**: 104,224 atrasos
3. **DFW (Dallas/Fort Worth International)**: 83,212 atrasos
4. **LAX (Los Angeles International)**: 81,895 atrasos
5. **DEN (Denver International)**: 71,443 atrasos

 **Conclusão:**
 Os aeroportos **Hartsfield-Jackson Atlanta** (ATL) e **Chicago O'Hare** (ORD) estão entre os mais críticos, com o maior número de atrasos tanto na decolagem quanto na chegada. 

## **Conclusão**

Neste projeto, realizei uma análise detalhada dos **atrasos de voos nos Estados Unidos**, utilizando dados de companhias aéreas, aeroportos e características dos voos. O trabalho envolveu a **coleta de dados**, **transformações no formato de Data Lake**, **modelagem de dados**, e **análises exploratórias**.

As principais perguntas de negócio foram respondidas com base nos dados disponíveis:

- **Quais companhias aéreas apresentam maior proporção de voos com atraso?**
- **Qual é o percentual de voos cancelados por companhia aérea?**
- **A distância ou o tempo de voo influenciam o atraso na chegada?**
- **Quais aeroportos concentram mais atrasos na decolagem e/ou chegada?**

As visualizações geradas, juntamente com as análises das proporções de atrasos e cancelamentos, permitiram identificar padrões e destacar as companhias e aeroportos com maior ou menor desempenho.

**Conclusões principais:**
- Algumas companhias aéreas têm uma proporção significativa de voos com atraso, como **NK** e **F9**.
- O percentual de voos cancelados também variou bastante, com **MQ** apresentando o maior percentual de cancelamentos.
- **A distância e o tempo de voo** não apresentaram correlações fortes com os atrasos na chegada.
- **Aeroportos como OAK e MSY** concentram o maior número de atrasos, indicando problemas operacionais nesses hubs.

## **Autoavaliação**

Durante a execução deste projeto, busquei **aplicar boas práticas de engenharia de dados** e **análises exploratórias**, seguindo todos os requisitos propostos para alcançar a **nota máxima**.

**Aspectos positivos:**
- **Coleta de dados e integração**: Utilizei dados do Kaggle e realizei uma integração eficiente entre diferentes fontes (flights, airports e airlines).
- **ETL eficiente**: Organizei o processo de transformação e carga de dados de maneira estruturada, usando **Data Lake** para armazenar os dados.
- **Análises claras e visualizações**: Concluí as perguntas de negócios com análises detalhadas e visualizações significativas, facilitando a interpretação dos resultados.

**Áreas para melhoria:**
- **Eficiência de código**: Algumas etapas poderiam ser otimizadas para evitar múltiplas operações de leitura/gravação no Databricks.
- **Expansão da análise**: A análise poderia incluir mais variáveis, como dados climáticos, que podem influenciar os atrasos.

**Conclusão da autoavaliação:**
Acredito que o projeto atendeu aos objetivos propostos, cumprindo os requisitos e gerando resultados relevantes. No entanto, sempre há espaço para melhorar, e continuar explorando variáveis adicionais poderia enriquecer ainda mais a análise.
